# Environmental Monitoring — Training Notebook
This notebook generates synthetic data, trains an AQI regressor and AQI classifier, and saves models to `model/`.


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
import joblib
import os


In [2]:
df = pd.read_csv(r'/mnt/data/smart_env_project/environment.csv')
df.head()

In [3]:
features = ['pm2_5','pm10','no2','co','o3','so2','temp','humidity','wind_speed']
X = df[features]
y_reg = df['aqi']
y_cls = df['aqi_category']
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y_reg, test_size=0.2, random_state=42)
scaler_r = StandardScaler()
X_train_r_s = scaler_r.fit_transform(X_train_r)
X_test_r_s = scaler_r.transform(X_test_r)


In [4]:
reg = RandomForestRegressor(n_estimators=150, max_depth=18, random_state=42, n_jobs=-1)
reg.fit(X_train_r_s, y_train_r)
preds_r = reg.predict(X_test_r_s)
print('R2:', r2_score(y_test_r, preds_r))
print('RMSE:', mean_squared_error(y_test_r, preds_r, squared=False))
os.makedirs('model', exist_ok=True)
joblib.dump(reg, 'model/aqi_regressor.pkl')
joblib.dump(scaler_r, 'model/scaler_reg.pkl')
print('Saved regressor and scaler')


In [5]:
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_cls, test_size=0.2, random_state=42, stratify=y_cls)
scaler_c = StandardScaler()
X_train_c_s = scaler_c.fit_transform(X_train_c)
X_test_c_s = scaler_c.transform(X_test_c)
clf = RandomForestClassifier(n_estimators=150, max_depth=18, random_state=42, n_jobs=-1)
clf.fit(X_train_c_s, y_train_c)
preds_c = clf.predict(X_test_c_s)
print('Accuracy:', accuracy_score(y_test_c, preds_c))
print(classification_report(y_test_c, preds_c))
joblib.dump(clf, 'model/aqi_classifier.pkl')
joblib.dump(scaler_c, 'model/scaler_clf.pkl')
print('Saved classifier and scaler')


In [6]:
iso = IsolationForest(n_estimators=200, contamination=0.02, random_state=42)
iso.fit(X)
joblib.dump(iso, 'model/anomaly_detector.pkl')
print('Saved anomaly detector')
